### Example 1: The Rearranging Algorithm
Run the rearranging simulator from loading the atomic array.

In [1]:
import os
import time
import argparse
from functools import wraps
import h5py
from tqdm import tqdm

# Rearranging algorithm imports
from lib.TrapGeometry.geometry import GeometeryGenerator
from lib.RearrangingAlgo.Path import PathGeneratorSingle, PathGeneratorParallel
from lib.RearrangingAlgo.Evolution import Simulation
import networkx as nx

# Rearranging pulse imports
from lib.Tweezer_control_software.AOD.signal.waveform_gen import (static_wave_gen, 
                                                                precal_tweezer_wave_gen, 
                                                                precal_alignment_wave_gen, 
                                                                cal_parallel_tweezer_wave_gen)

from utils import check_folder, reformat_path

#### The example code for running the rearranging path solver.
APAS Algorithm in this case.

We begin by setting up the enviornment for thr simulator.

In [2]:
# Number of target traps in the geometry
num_targets = 4
mode = 'rectangular'

Resolver result will be a list of rearranging path, and the state of the atoms in each time step in the form of pictures.

In [3]:
save_path = "./Experiments/"
check_folder(save_path)
save_dir = os.path.join(save_path, f'algorithm/Lattice_{mode}_NumTargets_{num_targets}_{time.strftime("%Y%m%d-%H%M%S")}/')# 
check_folder(os.path.join(save_dir, 'img/'))

Running the simulator

In [4]:
# Setup the entire trap geometry
atom_points, target_trap_points, reservior_trap_points, total_trap_points, graph, cost = GeometeryGenerator(num_targets = num_targets).rectangular_lattice()
        
# Take geometries and generate non-collision path
assembing_start = time.perf_counter()
path_total = PathGeneratorSingle(atom_points,
                                target_trap_points, 
                                reservior_trap_points, 
                                total_trap_points, 
                                graph, 
                                cost).lsap_general_lattice()#lsap_rectangular_lattice()
assembing_end = time.perf_counter()

distance_of_moves, num_of_moves, total_rearranging_time = Simulation(atom_points, 
                                                                    target_trap_points, 
                                                                    reservior_trap_points, 
                                                                    total_trap_points, 
                                                                    path_total, save_dir, 
                                                                    plot_mode=True, 
                                                                    tweezer_mode='single').start() # multi single

Target trap position assigned.

Reservior trap position assigned.

Atom distributed.

Path libaray generating...


100%|██████████| 4/4 [00:00<00:00, 15491.43it/s]


Path reordering...


100%|██████████| 3/3 [00:00<00:00, 19722.43it/s]


Plotting single-agent simulation...


  0%|          | 0/4 [00:00<?, ?it/s]

Now we can see in the "./Experiments/" folder, there will be a folder that contain the solution for this example. And here are some benchmark analysis for the algorithm. We will come back to this later.

In [ ]:
# Some benchmark outputs
print(f'Total travelling sites: {distance_of_moves}')
print(f'Total number of moves: {num_of_moves}')
print(f'Path calculating time: {(assembing_end - assembing_start) * 1000} ms')
print(f'Estimated Assembling time: {total_rearranging_time} \u03BCs') # microsec

Total travelling sites: 5
Total number of moves: 3
Path calculating time: 11.483207999845035 ms
Estimated Assembling time: 2999.7 μs


### Example 2: Generating The Rearranging Pulses
We can now play with the actual rearranging pulses for a given trap array. They will be stored in the form of a HDF5 file.

In [ ]:
# Number of target traps in the geometry
num_targets = 400
mode = 'rectangular'
atom_points, target_trap_points, reservior_trap_points, total_trap_points, freq_graph, cost = GeometeryGenerator(num_targets).rectangular_lattice()
save_dir = f'./Experiments/Physical_Moving/Lattice_{mode}_NumTargets_{num_targets}_{time.strftime("%Y%m%d-%H%M%S")}/'

Target trap position assigned.

Reservior trap position assigned.

Atom distributed.



In [ ]:
## Precalculate the data
print(f'Precalculating the signals')
awg_resolution = 625e06 # 625e3
grab_drop_time = 5e-6 #s
ave_velocity = 1e-1
trap_spacing = 5e-6
file_time = precal_tweezer_wave_gen(freq_graph, mode, awg_resolution, grab_drop_time, ave_velocity, trap_spacing,  save_folder = "./Experiments/")
print(f'Done.\n')

Precalculating the signals
Total trap numbers: 900
Total allowed hopping in this graph: 3480
Generating static signals...
Done

Generating grab and drop signals...


100%|██████████| 900/900 [00:00<00:00, 3285.44it/s]


Done

Generating moving signals...


100%|██████████| 3480/3480 [00:05<00:00, 625.46it/s]


Done

Moving calculation time: 5.56498384475708s, Average: 1.0537746344929144 ms
Done.



### Example 3: Readout Signal

In [ ]:
precal_data_path = "Experiments/AWG/precal_signals/MovingTweezerSignal_rectangular_20231108_143136.hdf5"
precal_data = h5py.File(precal_data_path, 'r')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'Experiments/AWG/precal_signals/MovingTweezerSignal_rectangular_20231108_143136.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

Signal length $=$ AWG_sample_rate $*$ signal_time 

In [ ]:
# Extract signal mode, can be "static", "grab", "drop", "move"
memory_usage = 0
for signal_mode in ["static", "grab", "drop", "move"]:
    print(f"Loading Mode: {signal_mode} \n")
    
    # Extract signal index with key, in the form of nodes pairs
    signals = list(precal_data[signal_mode]["signal"]) # signals is a 2xt np.array
    print(f"Number of signals: {len(signals)}\n")             
    
    # From the node pairs, extract the signal data
    for signal in tqdm(signals):
        signal_dataset = precal_data[signal_mode]["signal"][signal][()]
        data_channel_0 = signal_dataset[0].astype('float16') 
        data_channel_1 = signal_dataset[1].astype('float16') 
        print(f"Signal for: {signal}")
        print(f"Signal for DAC channel 0: {data_channel_0}, Memory usage: {data_channel_0.nbytes/(1024)**2} MB")
        print(f"Signal for DAC channel 1: {data_channel_1}, Memory usage: {data_channel_1.nbytes/(1024)**2} MB\n")
        memory_usage += (data_channel_0.nbytes + data_channel_1.nbytes)
print(f'Total memory usage: {memory_usage/(1024**2)} MB')


Loading Mode: static 

Number of signals: 1



100%|██████████| 1/1 [00:00<00:00, 116.04it/s]


Signal for: static
Signal for DAC channel 0: [ 0.254    0.08673 -0.1644  ... -0.0306   0.05884  0.0985 ], Memory usage: 1.1920928955078125 MB
Signal for DAC channel 1: [ 0.254    0.08673 -0.1644  ... -0.0306   0.05884  0.0985 ], Memory usage: 1.1920928955078125 MB

Loading Mode: grab 

Number of signals: 256



100%|██████████| 256/256 [00:00<00:00, 2113.66it/s]


Signal for: (70.0, 70.0)
Signal for DAC channel 0: [ 0.000e+00  3.177e-05 -4.667e-05 ...  1.455e-01  2.424e-01  2.241e-01], Memory usage: 0.0059604644775390625 MB
Signal for DAC channel 1: [ 0.000e+00  3.177e-05 -4.667e-05 ...  1.455e-01  2.424e-01  2.241e-01], Memory usage: 0.0059604644775390625 MB

Signal for: (70.0, 71.33333)
Signal for DAC channel 0: [ 0.000e+00  3.177e-05 -4.667e-05 ...  1.455e-01  2.424e-01  2.241e-01], Memory usage: 0.0059604644775390625 MB
Signal for DAC channel 1: [-0.000e+00  2.182e-05  1.341e-04 ...  1.841e-01  2.774e-02 -1.425e-01], Memory usage: 0.0059604644775390625 MB

Signal for: (70.0, 72.66667)
Signal for DAC channel 0: [ 0.000e+00  3.177e-05 -4.667e-05 ...  1.455e-01  2.424e-01  2.241e-01], Memory usage: 0.0059604644775390625 MB
Signal for DAC channel 1: [-0.0000e+00  4.8876e-06  1.1396e-04 ... -9.0866e-03  1.5991e-01
  2.4731e-01], Memory usage: 0.0059604644775390625 MB

Signal for: (70.0, 74.0)
Signal for DAC channel 0: [ 0.000e+00  3.177e-05 -4.66

...

Signal for: ((76.66667, 74.0), (76.66667, 75.33333))
Signal for DAC channel 0: [0.115  0.2372 0.2252 ... 0.0646 0.2146 0.2433], Memory usage: 0.059604644775390625 MB
Signal for DAC channel 1: [-0.1221 -0.2375 -0.2275 ... -0.0815 -0.2216 -0.2406], Memory usage: 0.059604644775390625 MB

Signal for: ((76.66667, 74.0), (78.0, 74.0))
Signal for DAC channel 0: [ 0.115    0.2372   0.2252  ...  0.184    0.01064 -0.1688 ], Memory usage: 0.059604644775390625 MB
Signal for DAC channel 1: [-0.1221  -0.2375  -0.2275  ...  0.2472   0.2073   0.05792], Memory usage: 0.059604644775390625 MB

Signal for: ((76.66667, 75.33333), (75.33333, 75.33333))
Signal for DAC channel 0: [ 0.115   0.2372  0.2252 ... -0.2438 -0.2152 -0.0689], Memory usage: 0.059604644775390625 MB
Signal for DAC channel 1: [-0.0823    0.1024    0.2311   ...  0.25      0.18      0.011635], Memory usage: 0.059604644775390625 MB

Signal for: ((76.66667, 75.33333), (76.66667, 74.0))
Signal for DAC channel 0: [0.115  0.2372 0.2252 ... 0.064

 81%|████████  | 779/960 [00:00<00:00, 1555.05it/s]

Signal for: ((83.33333, 75.33333), (84.66667, 75.33333))
Signal for DAC channel 0: [-0.0914  -0.2341  -0.2219  ...  0.2051   0.2427   0.11475], Memory usage: 0.059604644775390625 MB
Signal for DAC channel 1: [-0.0823    0.1024    0.2311   ...  0.25      0.18      0.011635], Memory usage: 0.059604644775390625 MB

Signal for: ((83.33333, 76.66667), (82.0, 76.66667))
Signal for DAC channel 0: [-0.0914   -0.2341   -0.2219   ...  0.001238 -0.1827   -0.2494  ], Memory usage: 0.059604644775390625 MB
Signal for DAC channel 1: [0.115  0.2372 0.2252 ... 0.0646 0.2146 0.2433], Memory usage: 0.059604644775390625 MB

Signal for: ((83.33333, 76.66667), (83.33333, 75.33333))
Signal for DAC channel 0: [-0.0914 -0.2341 -0.2219 ... -0.222  -0.0631  0.1376], Memory usage: 0.059604644775390625 MB
Signal for DAC channel 1: [ 0.115   0.2372  0.2252 ... -0.2438 -0.2152 -0.0689], Memory usage: 0.059604644775390625 MB

Signal for: ((83.33333, 76.66667), (83.33333, 78.0))
Signal for DAC channel 0: [-0.0914 -0.2

100%|██████████| 960/960 [00:00<00:00, 1558.06it/s]

Signal for: ((90.0, 79.33333), (88.66667, 79.33333))
Signal for DAC channel 0: [ 0.11816 -0.1002  -0.2421  ... -0.2134  -0.2354  -0.0824 ], Memory usage: 0.059604644775390625 MB
Signal for DAC channel 1: [ 0.1143  -0.0793  -0.2251  ... -0.2422  -0.2135  -0.05606], Memory usage: 0.059604644775390625 MB

Signal for: ((90.0, 79.33333), (90.0, 78.0))
Signal for DAC channel 0: [ 0.11816 -0.1002  -0.2421  ... -0.01624  0.1862   0.2462 ], Memory usage: 0.059604644775390625 MB
Signal for DAC channel 1: [ 0.1143  -0.0793  -0.2251  ...  0.1674  -0.01269 -0.1853 ], Memory usage: 0.059604644775390625 MB

Signal for: ((90.0, 79.33333), (90.0, 80.66667))
Signal for DAC channel 0: [ 0.11816 -0.1002  -0.2421  ... -0.01624  0.1862   0.2462 ], Memory usage: 0.059604644775390625 MB
Signal for DAC channel 1: [ 0.1143  -0.0793  -0.2251  ...  0.05795  0.2162   0.2399 ], Memory usage: 0.059604644775390625 MB

Signal for: ((90.0, 80.66667), (88.66667, 80.66667))
Signal for DAC channel 0: [ 0.11816 -0.1002  -0